In [2]:
# Biblioteca
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
import numpy as np


In [3]:
# Transformación: convertir a tensor y normalizar
transform = transforms.Compose([
    transforms.ToTensor(),  # Escala [0,255] → [0,1]
])

# Carga de datos
train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform, download=True)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64)


In [4]:
class DigitClassifier(nn.Module):
    def __init__(self):
        super(DigitClassifier, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)

    def forward(self, x):
        x = x.view(-1, 28 * 28)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return self.fc3(x)


In [5]:
model = DigitClassifier()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

# Entrenamiento
for epoch in range(30):
    for images, labels in train_loader:
        outputs = model(images)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}: Loss={loss.item():.4f}")


Epoch 1: Loss=0.1246
Epoch 2: Loss=0.0821
Epoch 3: Loss=0.0321
Epoch 4: Loss=0.1201
Epoch 5: Loss=0.0224
Epoch 6: Loss=0.0022
Epoch 7: Loss=0.1362
Epoch 8: Loss=0.0640
Epoch 9: Loss=0.0042
Epoch 10: Loss=0.0113
Epoch 11: Loss=0.0350
Epoch 12: Loss=0.0051
Epoch 13: Loss=0.0006
Epoch 14: Loss=0.0005
Epoch 15: Loss=0.0055
Epoch 16: Loss=0.0114
Epoch 17: Loss=0.0609
Epoch 18: Loss=0.0011
Epoch 19: Loss=0.0107
Epoch 20: Loss=0.0008
Epoch 21: Loss=0.2395
Epoch 22: Loss=0.0740
Epoch 23: Loss=0.0025
Epoch 24: Loss=0.0000
Epoch 25: Loss=0.0001
Epoch 26: Loss=0.0049
Epoch 27: Loss=0.0008
Epoch 28: Loss=0.0045
Epoch 29: Loss=0.0014
Epoch 30: Loss=0.0008


In [6]:
def quantize_tensor(tensor, scale=1000):
    return (tensor.detach().cpu().numpy() * scale).astype("int16")

quantized_weights = {}
for name, param in model.named_parameters():
    quantized_weights[name] = quantize_tensor(param.data)


In [7]:
quantized_weights

{'fc1.weight': array([[-23, -20, -22, ...,  13,   0, -19],
        [  0, -14, -11, ...,  12, -19,  33],
        [ 17, -19,  -2, ...,  -2,  35, -27],
        ...,
        [-22, -20,  24, ...,  34,  26, -25],
        [-14, -28, -15, ...,   0, -32,   3],
        [ -4, -11,  17, ..., -29, -28,  10]], dtype=int16),
 'fc1.bias': array([  76,  -54,  -45,   78,  183,   87,  -61,  155,  -69,   63, -111,
        -258,   71,  -96, -216,   61, -230,    0, -229,  -75,  114,  227,
        -252, -127,   81,  246,   69,  128, -318, -153,  153,   97, -137,
         228,  -35,   13,   14,  -84,  -55,  -57,  -62, -143,   95,  111,
        -110,   67,  -15,  122,   68,  312,  147,   81, -224, -110,  296,
        -109,   95, -238,  -41,   29,   75, -107,  319,  168,  157,  179,
         156,   82,   11, -175,  115,  265, -110, -192,  -31,  -10,  -40,
         -55,  -23,  159,   69, -205,  115,   45,   98, -189,  -53, -186,
         -54,  -83,   63,   48,   66, -174,  178,   76,   87,   98, -133,
          

In [ ]:
def export_weights_to_c(weights_dict, filename, scale=1000):
    with open(filename, "w") as f:
        f.write(f"#include <stdint.h>\n")
        for name, weights in weights_dict.items():
            flat_weights = weights.flatten()
            f.write(f"// {name}\n")
            f.write(f"const int16_t {name.replace('.', '_')}[{len(flat_weights)}] = {{\n")
            f.write(", ".join(map(str, flat_weights)))
            f.write("\n};\n\n")

# Ejecuta esta función después de cuantizar
export_weights_to_c(quantized_weights, "model_weights.h")
